<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### BL92 Portfolio Analysis Class

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

### The use of the "Python 3.10, Numpy 1.26.4" kernel is recommended.

## Standardizing Tasks

Topics of interest include:

* Importing, visualizing the data
* `FinancialData` base class
* `BL92Portfolio` portfolio class

## Real Data

**_Historical end-of-day financial time series data._**

Data retrieved vom Refinitiv Eikon for the German DAX 30 constituents.

The data sets:

    http://hilpisch.com/dax_eikon_eod_data.csv
    http://hilpisch.com/dax_eikon_mc_data.csv

## Imports and Data

In [ ]:
!git clone https://github.com/tpq-classes/python_for_asset_management.git
import sys
sys.path.append('python_for_asset_management')


In [ ]:
from fix_cufflink import *
cf_colors.to_rgba = fixed_to_rgba
cf_plotlytools.to_rgba = fixed_to_rgba

In [ ]:
!pip install eikon

In [ ]:
import math
import cufflinks
import eikon as ek
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')
cufflinks.set_config_file(offline=True)
np.set_printoptions(suppress=True, precision=4)
pd.options.display.float_format = '{:.5f}'.format
%config InlineBackend.figure_format = 'svg'

In [ ]:
url = 'http://hilpisch.com/dax_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True)

In [ ]:
raw.columns

In [ ]:
raw.iloc[:, :10].normalize().iplot()

## Financial Data Class

In [ ]:
class FinancialData:
    url = 'http://hilpisch.com/dax_eikon_eod_data.csv'
    url_ = 'http://hilpisch.com/dax_eikon_mc_data.csv'
    def __init__(self, universe):
        self.universe = universe
        self.no_assets = len(universe)
        self.retrieve_data()
        self.prepare_data()
    def retrieve_data(self):
        self.raw = pd.read_csv(self.url, index_col=0, parse_dates=True)
        self.raw_ = pd.read_csv(self.url_, index_col=0)
    def prepare_data(self):
        self.data = self.raw[self.universe]
        self.rets = np.log(self.data / self.data.shift(1))
        self.mc = (self.raw_.T[self.universe]).T
        self.mc['MC%'] = self.mc['MC'].apply(lambda x: x / self.mc['MC'].sum())
    def plot_data(self, cols=None):
        if cols is None:
            cols = self.universe
        self.data[cols].normalize().iplot()
    def plot_mc(self):
        self.mc.sort_values('MC').iplot(kind='pie',
                values='MC', labels='NAME', colorscale='rdylbu')
    def plot_corr(self):
        self.rets.corr().iplot(kind='heatmap', colorscale='reds')

In [ ]:
universe = raw.columns[:3]
universe

In [ ]:
fd = FinancialData(universe)

In [ ]:
fd.plot_data()

In [ ]:
fd.mc

In [ ]:
fd.plot_mc()

In [ ]:
fd.plot_corr()

## Views Object

In [ ]:
universe

In [ ]:
fd.rets.mean() * 252

In [ ]:
# views = [{'RIC': effect}, q, omega]

In [ ]:
views = [
    [{'1COV.DE': -1, 'ADSGn.DE': 1}, 0.05, 0.01],
    [{'ALVG.DE': 1}, 0.1, 0.02]
]

## BL92 Class

In [ ]:
from scipy.optimize import minimize

In [ ]:
class BL92Portfolio(FinancialData):
    def __init__(self, universe, views=None, tau=None):
        super().__init__(universe)
        self.equal_weights = self.no_assets * [1 / self.no_assets]
        self.mc_weights = self.mc['MC%']
        if views is None:
            self.views = list()
            self.tau = 0.000001
        else:
            self.views = views
            if tau is None:
                self.tau = 1
            else:
                self.tau = tau
        self.generate_bl_objects()
        self.generate_bl_statistics()
    def add_view(self, view):
        self.views.append(view)
        self.generate_bl_objects()
        self.generate_bl_statistics()
    def remove_view(self, index):
        self.views.pop(index)
        self.generate_bl_objects()
        self.generate_bl_statistics()
    def generate_bl_objects(self):
        v = len(self.views)
        self.P = pd.DataFrame(np.zeros((v, self.no_assets)), columns=self.universe)
        self.q = np.zeros(v)
        self.omega = np.zeros((v, v))
        for i, view in enumerate(self.views):
            for key, value in view[0].items():
                self.P.loc[i, key] = value
            self.q[i] = view[1]
            self.omega[i, i] = view[2]
    def generate_bl_statistics(self):
        self.mu = self.rets.mean() * 252
        self.cov = self.rets.cov() * 252
        C = self.tau * self.cov
        m1 = np.dot(self.P.T, np.dot(np.linalg.inv(self.omega), self.P))
        m1 += np.linalg.inv(C)
        m2 = np.dot(self.P.T, np.dot(np.linalg.inv(self.omega), self.q))
        m2 += np.dot(np.linalg.inv(C), self.mu)
        self.mu_ = np.dot(np.linalg.inv(m1), m2)
        self.cov_ = np.linalg.inv(m1) + self.cov
    def portfolio_return(self, weights):
        return np.dot(self.mu_, weights)
    def portfolio_variance(self, weights):
        return np.dot(weights, np.dot(self.cov_, weights))
    def portfolio_volatility(self, weights):
        return math.sqrt(self.portfolio_variance(weights))
    def portfolio_sharpe(self, weights):
        sharpe = self.portfolio_return(weights) / self.portfolio_volatility(weights)
        return sharpe
    def _set_bounds_constraints(self, bnds, cons):
        if bnds is None:
            self.bnds = self.no_assets * [(0, 1)]
        else:
            self.bnds = bnds
        if cons is None:
            self.cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}
        else:
            self.cons = cons
    def _get_results(self, opt, kind):
        ret = self.portfolio_return(opt['x'])
        vol = self.portfolio_volatility(opt['x'])
        sharpe = self.portfolio_sharpe(opt['x'])
        weights = pd.DataFrame(opt['x'], index=self.universe, columns=['weights',])
        res = {'kind': kind, 'weights': weights.round(7), 'return': ret,
               'volatility': vol, 'sharpe': sharpe}
        return res
    def minimum_volatility_portfolio(self, bnds=None, cons=None):
        self._set_bounds_constraints(bnds, cons)
        opt = minimize(self.portfolio_volatility, self.equal_weights,
                      bounds=self.bnds, constraints=self.cons)
        self.results = self._get_results(opt, 'Minimum Volatility')
        return self.results
    def maximum_sharpe_portfolio(self, bnds=None, cons=None):
        self._set_bounds_constraints(bnds, cons)
        tf = lambda weights: -self.portfolio_sharpe(weights)
        opt = minimize(tf, self.equal_weights, bounds=self.bnds,
                       constraints=self.cons)
        self.results = self._get_results(opt, 'Maximum Sharpe')
        return self.results
    def plot_weights(self, kind='pie'):
        if kind == 'pie':
            nonzero = self.results['weights'] > 0
            to_plot = self.results['weights'][nonzero['weights']].copy()
            to_plot['names'] = to_plot.index
            to_plot.iplot(kind='pie', values='weights',
                          labels='names', colorscale='rdylbu',
                          title='Optimal Weights | ' + self.results['kind'])
        else:
            self.results['weights'].iplot(kind='bar',
                    title='Optimal Weights | ' + self.results['kind'])
    def plot_performance(self):
        perf = (self.results['return'], self.results['volatility'], self.results['sharpe'])
        index = ['return', 'volatility', 'sharpe']
        to_plot = pd.DataFrame(perf, index=index, columns=['metrics',])
        to_plot.iplot(kind='bar', title='Performance Metrics  | ' + self.results['kind'])

In [ ]:
bl = BL92Portfolio(universe, views)

In [ ]:
# bl = BL92Portfolio(universe)

In [ ]:
# bl = BL92Portfolio(universe, views=[])

In [ ]:
# bl = BL92Portfolio(universe, views=[], tau=0.000001)

In [ ]:
bl.P

In [ ]:
bl.q

In [ ]:
bl.omega

In [ ]:
bl.mu

In [ ]:
bl.mu_

In [ ]:
bl.cov

In [ ]:
bl.cov_

In [ ]:
bl.cov_ - bl.cov

In [ ]:
bl.equal_weights

In [ ]:
bl.mc_weights

In [ ]:
bl.portfolio_return(bl.equal_weights)

In [ ]:
bl.portfolio_volatility(bl.equal_weights)

In [ ]:
bl.portfolio_sharpe(bl.equal_weights)

In [ ]:
bl.portfolio_return(bl.mc_weights)

In [ ]:
bl.portfolio_volatility(bl.mc_weights)

In [ ]:
bl.portfolio_sharpe(bl.mc_weights)

## Adding View 

In [ ]:
bl = BL92Portfolio(universe, views)

In [ ]:
bl.views

In [ ]:
view = [{'1COV.DE': 1}, 0.1, 0.000001]

In [ ]:
bl.add_view(view)

In [ ]:
bl.views

In [ ]:
bl.mu_

In [ ]:
bl.portfolio_sharpe(bl.equal_weights)

In [ ]:
bl.portfolio_sharpe(bl.mc_weights)

In [ ]:
bl.remove_view(-1)

In [ ]:
bl.views

## Optimal Portfolios 

In [ ]:
bl = BL92Portfolio(universe, views)

### Minimum Volatility

In [ ]:
res = bl.minimum_volatility_portfolio()

In [ ]:
res['weights']

In [ ]:
bl.plot_weights()

In [ ]:
bl.plot_performance()

### Maximum Sharpe Ratio 

In [ ]:
res = bl.maximum_sharpe_portfolio()

In [ ]:
res['weights']

In [ ]:
bl.plot_weights()

In [ ]:
bl.plot_performance()

### Bounded Maximum Sharpe

In [ ]:
bnds = bl.no_assets * [(0.025, 0.9)]

In [ ]:
res = bl.maximum_sharpe_portfolio(bnds=bnds)

In [ ]:
res['weights']

In [ ]:
bl.plot_weights()

In [ ]:
bl.plot_performance()

### Short Sales Allowed

In [ ]:
bnds = bl.no_assets * [(-1, 1)]

In [ ]:
res = bl.maximum_sharpe_portfolio(bnds=bnds, cons={})

In [ ]:
res['weights']

In [ ]:
bl.plot_weights()

In [ ]:
bl.plot_performance()

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>